In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

import time

In [11]:
# Configure Selenium WebDriver
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode (optional)

# Use webdriver_manager to handle driver download
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)


In [3]:
import pandas as pd
df=pd.read_csv(r'C:\Users\Anis\Desktop\Crispr model\off_target\blast_results.csv')

In [4]:
df.head()

,Label,HD Text,AR Text
0,"Homo sapiens isolate CHM13 chromosome 3, alter...",Range 1: 193549069 to 193549087GenBankGraphics...,Query 3 GCAGTAGCTAGCTAGCTAG 21\n ...
1,"Homo sapiens chromosome 3, GRCh38.p14 Primary ...",Range 2: 81734675 to 81734691GenBankGraphics\n...,Query 4 CAGTAGCTAGCTAGCTA 20\n ...
2,"Homo sapiens isolate CHM13 chromosome 2, alter...",Range 3: 1015481 to 1015495GenBankGraphics\nNe...,Query 7 TAGCTAGCTAGCTAG 21\n ...
3,"Homo sapiens isolate CHM13 chromosome 6, alter...",Range 4: 2872649 to 2872663GenBankGraphics\nNe...,Query 7 TAGCTAGCTAGCTAG 21\n ...
4,"Homo sapiens isolate CHM13 chromosome 21, alte...",Range 5: 26308090 to 26308104GenBankGraphics\n...,Query 7 TAGCTAGCTAGCTAG 21\n ...


In [ ]:
print

In [54]:
print(df.loc[0,'HD Text'])


Range 1: 193549069 to 193549087GenBankGraphics
Next Match
Previous Match
Alignment statistics for match #1
Score Expect Identities Gaps Strand
38.2 bits(19) 0.084 19/19(100%) 0/19(0%) Plus/Minus
Features:
84024 bp at 5' side: interleukin-1 receptor accessory protein isoform x3
122539 bp at 3' side: geminin coiled-coil domain-containing protein 1


In [55]:
print(df.loc[0,'AR Text'])

Query  3          GCAGTAGCTAGCTAGCTAG  21
                  |||||||||||||||||||
Sbjct  193549087  GCAGTAGCTAGCTAGCTAG  193549069


In [5]:
import pandas as pd

# Read the original CSV file
df = pd.read_csv('blast_results.csv')

# Initialize lists to store the new columns
ranges = []
alignment_stats = []
features = []
queries = []
subjects = []

# Process each row to extract subcolumns
for index, row in df.iterrows():
    # Process HD Text
    hd_text = row['HD Text']
    hd_lines = hd_text.split('\n')

    # Extract Range
    range_info = hd_lines[0].replace('GenBankGraphics', '').strip()
    ranges.append(range_info)

    # Extract Alignment Statistics
    alignment_stat_info = hd_lines[3:6]
    alignment_stats.append(' '.join(alignment_stat_info).strip())

    # Extract Features
    features_info = hd_lines[7:]
    features.append(' '.join(features_info).strip())

    # Process AR Text
    ar_text = row['AR Text']
    ar_lines = ar_text.split('\n')

    # Extract Query and Subject
    query_info = ar_lines[0].strip()
    subject_info = ar_lines[2].strip()

    queries.append(query_info)
    subjects.append(subject_info)

# Create a new DataFrame with the extracted subcolumns
new_df = pd.DataFrame({
    'Range': ranges,
    'Alignment Statistics': alignment_stats,
    'Features': features,
    'Query': queries,
    'Subject': subjects
})

# Write the new DataFrame to a new CSV file
new_df.to_csv('blast_results_split.csv', index=False)

print("New CSV file 'blast_results_split.csv' created successfully.")


New CSV file 'blast_results_split.csv' created successfully.


In [6]:
df2=pd.read_csv(r'C:\Users\Anis\Desktop\Crispr model\off_target\blast_results_split.csv')

In [7]:
df2.head()

,Range,Alignment Statistics,Features,Query,Subject
0,Range 1: 193549069 to 193549087,Alignment statistics for match #1 Score Expect...,84024 bp at 5' side: interleukin-1 receptor ac...,Query 3 GCAGTAGCTAGCTAGCTAG 21,Sbjct 193549087 GCAGTAGCTAGCTAGCTAG 193549069
1,Range 2: 81734675 to 81734691,First Match Alignment statistics for match #2 ...,"Features: 1,4-alpha-glucan-branching enzyme",Query 4 CAGTAGCTAGCTAGCTA 20,Sbjct 81734675 CAGTAGCTAGCTAGCTA 81734691
2,Range 3: 1015481 to 1015495,First Match Alignment statistics for match #3 ...,Features: 617909 bp at 5' side: neural cell ad...,Query 7 TAGCTAGCTAGCTAG 21,Sbjct 1015481 TAGCTAGCTAGCTAG 1015495
3,Range 4: 2872649 to 2872663,First Match Alignment statistics for match #4 ...,Features: contactin-4 isoform x1 contactin-4 i...,Query 7 TAGCTAGCTAGCTAG 21,Sbjct 2872663 TAGCTAGCTAGCTAG 2872649
4,Range 5: 26308090 to 26308104,First Match Alignment statistics for match #5 ...,Features: 510830 bp at 5' side: 3-oxoacyl-[acy...,Query 7 TAGCTAGCTAGCTAG 21,Sbjct 26308104 TAGCTAGCTAGCTAG 26308090


In [8]:
import pandas as pd
import re

# Read the original CSV file
df = pd.read_csv(r'C:\Users\Anis\Desktop\Crispr model\off_target\blast_results.csv')

# Initialize lists to store the new columns
sequences = []
starts = []
ends = []
phenotypes = []

# Process each row to extract subcolumns
for index, row in df.iterrows():
    # Process HD Text
    hd_text = row['HD Text']
    hd_lines = hd_text.split('\n')

    # Extract Range and Phenotypes
    range_info = hd_lines[0]
    feature_lines = hd_lines[-2:]  # Assuming phenotypes are in the last two lines

    # Extract start and end from the range
    range_match = re.search(r'Range \d+: (\d+) to (\d+)', range_info)
    if range_match:
        start = range_match.group(1)
        end = range_match.group(2)
        starts.append(start)
        ends.append(end)
    else:
        starts.append(None)
        ends.append(None)

    # Extract phenotypes
    phenotypes_info = [line.split(': ')[1] for line in feature_lines if ': ' in line]
    phenotypes.append(', '.join(phenotypes_info))

    # Process AR Text
    ar_text = row['AR Text']
    ar_lines = ar_text.split('\n')

    # Extract Sequence
    sequence_info = ar_lines[0].strip()
    sequence_match = re.search(r'Query \d+\s+(\w+)\s+\d+', sequence_info)
    if sequence_match:
        sequence = sequence_match.group(1)
        sequences.append(sequence)
    else:
        sequences.append(None)

# Create a new DataFrame with the extracted subcolumns
new_df = pd.DataFrame({
    'sequence': sequences,
    'start': starts,
    'end': ends,
    'phenotypes': phenotypes
})

# Write the new DataFrame to a new CSV file
new_df.to_csv('blast_results_split.csv', index=False)

print("New CSV file 'blast_results_split.csv' created successfully.")


New CSV file 'blast_results_split.csv' created successfully.


In [61]:
new_df.head()

,sequence,start,end,phenotypes
0,None,193549069,193549087,interleukin-1 receptor accessory protein isofo...
1,None,81734675,81734691,
2,None,1015481,1015495,neural cell adhesion molecule l1-like protein ...
3,None,2872649,2872663,
4,None,26308090,26308104,"3-oxoacyl-[acyl-carrier-protein] synthase, mit..."


In [10]:
import pandas as pd
import re

# Read the original CSV file
df = pd.read_csv(r'C:\Users\Anis\Desktop\Crispr model\off_target\blast_results.csv')

# Initialize lists to store the new columns
chromosome_location=[]
sequences = []
starts = []
ends = []
phenotypes = []

# Process each row to extract subcolumns
for index, row in df.iterrows():
    # Process HD Text
    hd_text = row['HD Text']
    hd_lines = hd_text.split('\n')

    # Extract Range and Phenotypes
    range_info = hd_lines[0]
    feature_lines = hd_lines[-2:]  # Assuming phenotypes are in the last two lines

    # Extract start and end from the range
    range_match = re.search(r'Range \d+: (\d+) to (\d+)', range_info)
    if range_match:
        start = range_match.group(1)
        end = range_match.group(2)
        starts.append(start)
        ends.append(end)
    else:
        starts.append(None)
        ends.append(None)

    # Extract phenotypes
    phenotypes_info = [line.split(': ')[1] for line in feature_lines if ': ' in line]
    phenotypes.append(', '.join(phenotypes_info))

    # Process AR Text
    ar_text = row['AR Text']
    ar_lines = ar_text.split('\n')

    # Extract Sequence
    query_line = ar_lines[0].strip()
    query_match = re.search(r'Query\s+\d+\s+([A-Z]+)\s+\d+', query_line)
    if query_match:
        sequence = query_match.group(1)
        sequences.append(sequence)
    else:
        sequences.append(None)
    #chromosome location
    chromosome_location.append(row['Label'])


# Create a new DataFrame with the extracted subcolumns
new_df = pd.DataFrame({
    'chromosome location': chromosome_location,
    'sequence': sequences,
    'start': starts,
    'end': ends,
    'phenotypes': phenotypes
})

# Write the new DataFrame to a new CSV file
new_df.to_csv('blast_results_split.csv', index=False)

print("New CSV file 'blast_results_split.csv' created successfully.")


New CSV file 'blast_results_split.csv' created successfully.


In [63]:
df3=pd.read_csv(r'C:\Users\Anis\Desktop\Crispr model\off_target\blast_results_split.csv')

In [64]:
df3.head()

,chromosome location,sequence,start,end,phenotypes
0,"Homo sapiens isolate CHM13 chromosome 3, alter...",GCAGTAGCTAGCTAGCTAG,193549069,193549087,interleukin-1 receptor accessory protein isofo...
1,"Homo sapiens chromosome 3, GRCh38.p14 Primary ...",CAGTAGCTAGCTAGCTA,81734675,81734691,NaN
2,"Homo sapiens isolate CHM13 chromosome 2, alter...",TAGCTAGCTAGCTAG,1015481,1015495,neural cell adhesion molecule l1-like protein ...
3,"Homo sapiens isolate CHM13 chromosome 6, alter...",TAGCTAGCTAGCTAG,2872649,2872663,NaN
4,"Homo sapiens isolate CHM13 chromosome 21, alte...",TAGCTAGCTAGCTAG,26308090,26308104,"3-oxoacyl-[acyl-carrier-protein] synthase, mit..."


In [65]:
print(df3)

                                 chromosome location             sequence  \
0  Homo sapiens isolate CHM13 chromosome 3, alter...  GCAGTAGCTAGCTAGCTAG   
1  Homo sapiens chromosome 3, GRCh38.p14 Primary ...    CAGTAGCTAGCTAGCTA   
2  Homo sapiens isolate CHM13 chromosome 2, alter...      TAGCTAGCTAGCTAG   
3  Homo sapiens isolate CHM13 chromosome 6, alter...      TAGCTAGCTAGCTAG   
4  Homo sapiens isolate CHM13 chromosome 21, alte...      TAGCTAGCTAGCTAG   

       start        end                                         phenotypes  
0  193549069  193549087  interleukin-1 receptor accessory protein isofo...  
1   81734675   81734691                                                NaN  
2    1015481    1015495  neural cell adhesion molecule l1-like protein ...  
3    2872649    2872663                                                NaN  
4   26308090   26308104  3-oxoacyl-[acyl-carrier-protein] synthase, mit...  


second try

In [2]:
import csv
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Define the URL
url = "https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastSearch&BLAST_SPEC=OGP__9606__9558&LINK_LOC=blasthome"

# Set up the WebDriver (Make sure you have the appropriate WebDriver for your browser, e.g., Chrome, Firefox)
driver = webdriver.Chrome()

try:
    # Navigate to the URL
    driver.get(url)

    # Enter the sequence into the textarea
    sequence_textarea = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.ID, "seq"))
    )
    sequence_textarea.send_keys("ATGCAGTAGCTAGCTAGCTAG")  # Replace with your actual sequence

    # Click the "BLAST" button
    blast_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "//div[@id='blastButton1']/input[@class='blastbutton']"))
    )
    blast_button.click()

    # Wait until the results page is fully loaded
    format_organism_input = WebDriverWait(driver, 120).until(
        EC.presence_of_element_located((By.ID, "qorganism"))
    )

    # Click the link with ind="1"
    link_to_click = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "//tr[@ind='1']//a[@class='deflnDesc']"))
    )
    link_to_click.click()

    # Initialize lists for hd and ar divs and labels
    hd_texts = []
    ar_texts = []
    labels = []

    # Get all hd_ and ar_ divs
    hd_div_elements = WebDriverWait(driver, 30).until(
        EC.presence_of_all_elements_located((By.XPATH, "//div[starts-with(@id, 'hd_')]"))
    )
    ar_div_elements = WebDriverWait(driver, 30).until(
        EC.presence_of_all_elements_located((By.XPATH, "//div[starts-with(@id, 'ar_')]/pre"))
    )

    # Get all label divs
    label_div_elements = WebDriverWait(driver, 30).until(
        EC.presence_of_all_elements_located((By.XPATH, "//div[@class='dlfRow']"))
    )

    # Ensure the lists have the same length
    num_entries = min(len(hd_div_elements), len(ar_div_elements), len(label_div_elements))

    # Extract text from hd_ and ar_ divs and labels
    for i in range(num_entries):
        try:
            label_text = label_div_elements[i].text
            hd_texts.append(hd_div_elements[i].text)
            ar_texts.append(ar_div_elements[i].text)
            labels.append(label_text)
        except Exception as e:
            print(f"An error occurred while fetching entry {i}: {e}")
            break

    # Combine the texts and labels into pairs
    combined_texts = zip(labels, hd_texts, ar_texts)

    # Write to a CSV file
    with open('blast_results.csv', 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['Label', 'HD Text', 'AR Text'])  # Write headers
        for label, hd_text, ar_text in combined_texts:
            csvwriter.writerow([label, hd_text, ar_text])

finally:
    # Close the WebDriver
    time.sleep(10)  # Keep the browser open for 10 seconds for demonstration purposes
    driver.quit()
